# Code Summarization with Deep Reinforcement Learning
Paper[1] is linked: [Improving Code Summarization with Deep Reinforcement Learning](https://arxiv.org/abs/1811.07234)  
We're attempting to add concepts from Paper[2] [Unsupervised Translation of Programming Languages](https://arxiv.org/pdf/2006.03511)  
How? Not sure...  
I'm thinking of using their masking technique to finetune the product of Paper[1]. Essentially, with a given

# 1. Load Data

In [1]:
import pandas as pd
import os
cache_dir = 'F:\\.cache\\huggingface\\'
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir
import code_search_net
import datasets
import re
from nltk.stem import PorterStemmer
import ast
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
import math
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# You can use the below line instead of you don't want to load locally.
# BUt I havne't figured out to only choose certain datasets, I think its the uploader's fault
# Total is like 10+GB's
# d = datasets.load_dataset('code-search-net/code_search_net')

In [3]:
# What language you used for the data, otherwise default to all
dl_manager = datasets.DownloadManager('python')
csn = code_search_net.CodeSearchNet(config_name='python')
# You can remove the loc variable. My c:drive has no space so I did this instead.
generators = csn._split_generators(dl_manager, loc = 'F:\\.cache\\huggingface\\')

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
raw_data = []
for i, data in csn._generate_examples(generators[0].gen_kwargs['filepaths'][:1]):
    raw_data.append(data)

In [5]:
# Of interest:
# func_code_tokens
# func_documentation_tokens
# possibly func_path_in_repository?
print(len(raw_data))
raw_data[0].keys()


30000


dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])

# 2. Create Inputs
"To convert code into sequential text, we tokenize the code by `{. , ” ’ : ; ) ( ! (space)}`, which  has been used in [8].  
We tokenize the comment by {(space)}" - Paper

## 2a. Create AST
ASTs to binary trees by the following two steps which have been adopted in [28]:  
- a) Split nodes with more than 2 children, generate a new right child together with the old left child as its children, and then put all children except the leftmost as the children of this new node.   
Repeat this operation in a top-down way until only nodes with 0, 1, 2 children left;
- b) Combine nodes with 1 child with its child.  

(from the paper)

In [6]:
def parse_tree(lines):
    #replace the DCNL & DCSP symbols
    #need to deal with different indentations& spacings. The authors used DCSP for both '\t' & whitespace

    lines=lines.replace(' DCNL ',' \n')
    lines=lines.replace(' DCSP ',' \t')
    lines=lines.replace('DCSP ',' \t')

    #create temporary python file for every line (function) in input file
    tree = ast.parse(lines)
    temp_ast=ast.dump(tree)
    
    return temp_ast
def clean_data(line):

    clean = re.sub(r"""
            [,.;@#?!&$()='\\_`:>"%/{}*]+  # Accept one or more copies of punctuation
            \ *           # plus zero or more copies of a space,
            """,
            " ",          # and replace it with a single space
            line, flags=re.VERBOSE)
    #Manually handle cases not accepted by sub
    clean = clean.replace("[", "")
    clean = clean.replace("]", "")
    clean = clean.replace("-", "")
    # tokenize on white space
    line = clean.split()
    # convert to lower case
    line = [word.lower() for word in line]
    # store as string
    return line
    # remove empty strings

errors = []
for i, f in enumerate(raw_data):
    try:
        f['func_ast_string'] = parse_tree(f['func_code_string'])
        f['func_ast_tokens'] = clean_data(f['func_ast_string'])
    except Exception as e:
        #print(e)
        errors.append(i)

good_data = [x for i,x in enumerate(raw_data) if i not in errors]
print(len(good_data), len(errors), len(raw_data))


29969 31 30000


In [7]:
stemmer = PorterStemmer()
def token_code(func_code_string):
    stripped = re.sub('"""[\s\S]*"""', '', func_code_string.lower()) # apaprently code has the entire docstring in there too
    no_comment = re.sub('#.*', '', stripped) # We should base it off just the comments
    tokens = re.split('[ \[\]{}\.,:;\)\(!\)=(\\n)(\\t)"\']', no_comment) # split appropaite, then next line remove ''
    return [stemmer.stem(x) for x in filter(None, tokens)], no_comment # https://stackoverflow.com/questions/30933216/split-by-regex-without-resulting-empty-strings-in-python
print(token_code(raw_data[0]['func_code_string']))
def token_comm(func_doc_string):
    stripped = re.sub('[\[\](\\n)|,:\.─├│└(\\t)"\']', ' ', func_doc_string.lower())
    tokens = re.split(" ", stripped)
    return [stemmer.stem(x) for x in filter(None, tokens)] # https://stackoverflow.com/questions/30933216/split-by-regex-without-resulting-empty-strings-in-python
print(token_comm(raw_data[0]['func_documentation_string']))

(['def', 'train', 'train_dir', 'model_save_path', 'none', 'n_neighbor', 'none', 'knn_algo', 'ball_tre', 'verbos', 'fals', 'x', 'y', 'for', 'class_dir', 'in', 'os', 'listdir', 'train_dir', 'if', 'not', 'os', 'path', 'isdir', 'os', 'path', 'join', 'train_dir', 'class_dir', 'continu', 'for', 'img_path', 'in', 'image_files_in_fold', 'os', 'path', 'join', 'train_dir', 'class_dir', 'imag', 'face_recognit', 'load_image_fil', 'img_path', 'face_bounding_box', 'face_recognit', 'face_loc', 'imag', 'if', 'len', 'face_bounding_box', '1', 'if', 'verbos', 'print', 'imag', 'not', 'suitabl', 'for', 'train', 'format', 'img_path', 'didn', 't', 'find', 'a', 'face', 'if', 'len', 'face_bounding_box', '<', '1', 'els', 'found', 'more', 'than', 'one', 'face', 'els', 'x', 'append', 'face_recognit', 'face_encod', 'imag', 'known_face_loc', 'face_bounding_box', '0', 'y', 'append', 'class_dir', 'if', 'n_neighbor', 'is', 'none', 'n_neighbor', 'int', 'round', 'math', 'sqrt', 'len', 'x', 'if', 'verbos', 'print', 'chos

## 2b. Vocab Creation
- Reparse func_code_string into tokens
- Might respase func_documaetnation_string
- lower case

In [8]:
vocab_code = {'<SOS>' : 0, '<EOS>': 1} # Is this necessary?
max_code = 0
i = 1
vocab_comm = {'<SOS>' : 0, '<EOS>': 1}
max_comm = 0
j = 1
vocab_struct = {'<SOS>' : 0, '<EOS>': 1}
max_struc = 0
k = 1
for f in good_data:
    # I think we will need to preprocess out own toeksn btu for right now prrof of concenpt:
    f['func_code_tokens'], f['func_code_string'] = token_code(f['func_code_string']) # replace with out tokens
    f['func_code_tokens'] = ['<SOS>'] + f['func_code_tokens'] + ['<EOS>']
    
    for code_token in f['func_code_tokens']:
        if code_token not in vocab_code.keys():
            vocab_code[code_token] = i
            i += 1
    max_code = max(max_code, len(f['func_code_tokens']))
    # With code
    f['func_ast_tokens'] =  ['<SOS>'] + f['func_ast_tokens']  + ['<EOS>']
    for code_token in f['func_ast_tokens']:
        if code_token not in vocab_code.keys():
            vocab_code[code_token] = k
            k += 1
    max_struc = max(max_struc, len(f['func_ast_tokens']))
    # # Standalone
    # f['func_ast_tokens'] =  ['<SOS>'] + f['func_ast_tokens']  + ['<EOS>']
    # for code_token in f['func_ast_tokens']:
    #     if code_token not in vocab_struct.keys():
    #         vocab_struct[code_token] = k
    #         k += 1
    # max_struc = max(max_struc, len(f['func_ast_tokens']))
    f['func_documentation_tokens'] =  ['<SOS>'] + token_comm(f['func_documentation_string']) + ['<EOS>']
    for comm_token in f['func_documentation_tokens']:
        if comm_token not in vocab_comm.keys():
            vocab_comm[comm_token] = j
            j += 1
    max_comm = max(max_comm, len(f['func_documentation_tokens']))
print('Vocab / Max Sentence Length')
print(len(vocab_code), max_code, ' | ', len(vocab_comm), max_comm)


Vocab / Max Sentence Length
159032 6184  |  58199 4959


In [89]:
index_to_comm = {v: k for k, v in vocab_comm.items()} # https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping

## 2x Dataloader

In [54]:
# You can probalby do this faster with torch?
MAX_LENGTH = 128
input_seq = []
input_str = []
target = []
for x in good_data:
    temp_seq = []
    temp_str = []
    temp_comm = []
    for token in x['func_code_tokens']:
        temp_seq.append(vocab_code[token])
    for token in x['func_ast_tokens']:
        temp_str.append(vocab_code[token])
        #  temp_str.append(vocab_struct[token])
    for token in x['func_documentation_tokens']:
        temp_comm.append(vocab_comm[token])
    if len(temp_seq) > MAX_LENGTH or len(temp_str) > MAX_LENGTH or len(temp_comm) > MAX_LENGTH: # If too big we'll discard
        continue
    else: # Otherwise we'll padd them util same length
        for x in range(len(temp_seq), MAX_LENGTH):
            temp_seq.append(vocab_code['<EOS>'])
        for x in range(len(temp_str), MAX_LENGTH):
            temp_str.append(vocab_code['<EOS>'])
        # for x in range(len(temp_str), 512):
        #     temp_str.append(vocab_struct['<EOS>'])

        for x in range(len(temp_comm), MAX_LENGTH):
            temp_comm.append(vocab_comm['<EOS>'])
        input_seq.append(temp_seq)
        input_str.append(temp_str)
        target.append(temp_comm)
        
input_seq = torch.tensor(input_seq, dtype=torch.int).to(device)
input_str = torch.tensor(input_str, dtype=torch.int).to(device)
target = torch.tensor(target, dtype=torch.int).to(device)
target.shape

torch.Size([2390, 128])

In [55]:
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data import random_split
from torch import Generator
def create_dataloaders():
    all_data = torch.stack([input_seq, input_str, target], dim = 1)
    gen = Generator().manual_seed(818)
    batch_size = 64
    train_set, test_set = random_split(all_data, [0.7, 0.3], generator=gen)
    train_dataloader = DataLoader(train_set, sampler=RandomSampler(train_set), batch_size=batch_size)
    test_dataloader = DataLoader(test_set, sampler=RandomSampler(test_set), batch_size=batch_size)
    return train_dataloader, test_dataloader

In [57]:
train_dataloader, test_dataloader = create_dataloaders()
for x in train_dataloader:
    print(x[:,0,:].shape) # input_seq
    print(x[:,1,:].shape) # input_str
    print(x[:,2,:].shape) # target
    break

torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 128])


# 3. Model Definition
Probably hardest bit, I got as many variables from the paper as I could.  
i think a lot of it will be similar to what we did in class projects (Lab 5)

## 3a Fxn

In [70]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, code_vocab, struct_vocab):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        # these are probalby btached, investigate later
        # Create embeeddings
        self.enc_embedding = nn.Embedding(code_vocab, hidden_size)
        #self.enc_embedding_struct = nn.Embedding(struct_vocab, hidden_size)

        #encode with GRU
        self.enc_GRU_seq = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.enc_GRU_struct = nn.GRU(hidden_size, hidden_size, batch_first=True)
        # END ENCODE

    def forward(self, seq, struct):
        '''
        Takes Sequential and Structural representation of code and creates two hidden layers and outputs out of them.
        '''
        # e_seq = self.enc_embedding_seq(seq)
        # e_struct = self.enc_embedding_struct(struct)
        e_seq = self.enc_embedding(seq)
        e_struct = self.enc_embedding(struct)
        out_seq, hide_seq = self.enc_GRU_seq(e_seq)
        out_struct, hide_struct = self.enc_GRU_struct(e_struct)

        return out_seq, hide_seq, out_struct, hide_struct

def Attention(hidden, key, mask=None, dropout=None): # Key and Value are the? same
    # Goal: batch x embed x embed size
    #print('attention', hidden.shape, key.shape)
    # 1 x 512 * 512 x 512
    scores = torch.bmm(hidden.permute(1, 0, 2), key.permute(0,2,1) ) 
    attention = F.softmax(scores, dim = -1)
    # atn = 1 x 512, ie weights for each of the query
    # Context vector
    #print(attention.shape, hidden.permute(1, 0, 2).shape)
    return torch.bmm(attention, key), attention

class HybridAttention(nn.Module):
    def __init__(self, hidden_size):
        super(HybridAttention, self).__init__()
        # START DCODE
        self.linear1 = nn.Linear(hidden_size*2, hidden_size)
        self.linear2 = nn.Linear(hidden_size*2, hidden_size)

    def forward(self, enc_output, h_str, h_seq):
        # Attention of each hidden state we look at and get context vetcor
        d_str, a_str = Attention(h_str, enc_output)
        d_seq, a_seq = Attention(h_seq, enc_output)
        #print('contexts', d_str.shape, d_seq.shape)
        # Get respective context vectors and combine together
        # creats an aggregate contex vector.
        d_t = self.linear1(torch.cat([d_str, d_seq], dim = -1))
        #print('context vector', d_t.shape)
        
        return d_t
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, comm_vocab):
        super(AttnDecoder, self).__init__()
        self.dec_embedding = nn.Embedding(comm_vocab, hidden_size)
        self.attention = HybridAttention(hidden_size)
        self.tanh = nn.Tanh()
        self.dec_GRU = nn.GRU(2*hidden_size, hidden_size, batch_first=True)
        #self.dec_hid = nn.Linear(2*hidden_size, hidden_size)
        self.pred = nn.Linear(hidden_size,  comm_vocab, 1) # Predict 

    def forward(self, encoder_outputs, h_seq, h_struct):
        batch_size = encoder_outputs.size(0)
        
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(0)
        decoder_hidden = h_seq
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            embed = self.dec_embedding(decoder_input)
            #print('start', encoder_outputs.shape, h_seq.shape, h_struct.shape)
            context = self.attention(encoder_outputs, h_seq, h_struct)

            # what is supposedly an LSTM or something like that
            #s_squiggly = self.tanh(self.dec_hid(torch.cat(embed, context)))
            #output = self.pred(s_squiggly)
            #print(embed.shape, context.shape)
            output, decoder_hidden =  self.dec_GRU(torch.cat([embed, context], dim = -1), decoder_hidden)
            output = self.pred(output)

            _, topi = output.topk(1)
            decoder_input = topi.squeeze(-1).detach()  # detach from history as input
            decoder_outputs.append(output) 
    

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
     

        return decoder_outputs#, decoder_hidden


## 3b. Construct Actor Network

In [59]:
class Actor(nn.Module):
    def __init__(self, input_size, hidden_size, code_vocab, struct_vocab, comm_vocab):
        super(Actor, self).__init__()
        self.encoder = Encoder(input_size,hidden_size,code_vocab, struct_vocab)
        self.decoder = AttnDecoder(hidden_size,comm_vocab)
    def forward(self, code, struct):
        out_seq, hide_seq, out_struct, hide_struct = self.encoder(code, struct)
        
        decoder_outputs = self.decoder(out_seq, hide_seq, hide_struct)
        return decoder_outputs

In [61]:
# Taken from paper
batch_size = 64
hidden_size = 512
embedding_size = 512 #GRU, Layers
lr = 0.001
skip_num = 0
code_vocab_size = len(vocab_code)
struct_vocab_size = len(vocab_struct)
comment_vocab_size = len(vocab_comm)
#comment_max_len = max_comm # This and max_c

## 3c. Critic Network

## 3d. Training Loop?

In [62]:
train_dataloader, test_dataloader = create_dataloaders()


In [73]:
try:
    del ActorNetwork
    del loss_actor_pretrain
    del optim_actor
except:
    pass
ActorNetwork = Actor(128, hidden_size, code_vocab_size, struct_vocab_size, comment_vocab_size).to(device)
loss_actor_pretrain = nn.NLLLoss()
optim_actor = optim.Adagrad(ActorNetwork.parameters(), lr=lr)

In [95]:
from torcheval.metrics import Perplexity
try:
        del out
except:
        pass
for i in range(10):
        print(i)
        for train_in in train_dataloader:
                seq_in = train_in[:,0,:]
                struct_in =train_in[:,1,:]
                target_tensor = train_in[:,2,:]
                optim_actor.zero_grad()
                
                out = ActorNetwork(seq_in, struct_in)
                loss = loss_actor_pretrain(
                        out.view(-1, out.size(-1)),
                        target_tensor.long().reshape(-1)
                )
                loss.backward()
                del out
        torch.save({'epoch': i,
           'model_state_dict': ActorNetwork.state_dict(),
           'optimizer_state_dict': optim_actor.state_dict(),
           'loss': loss},f'models/actor_128_{i}')
        torch.save(ActorNetwork, f'models/actor_128_{i}.whole')
    

0
1
2
3
4


KeyboardInterrupt: 

In [93]:
def out_to_sentence(indexes):
    sentence = []
    #print(indexes)
    for index in indexes:
        sentence.append(index_to_comm[index.item()])
        if index == 1:
            break
    return sentence

In [97]:
print('predict', out_to_sentence(decoded_ids[0]))
print('target', out_to_sentence(target_tensor[0]))

predict ['png/svg', '``\\\\d``', '__call__', 'vpc-6b1fe402', '跟踪', 'port_by_scheme``', 'pragma', '__option', 'nat', 'active\r', 'commiss', 'device1', 'results\r', 'source_dir', 'account', '``na', 'emphasi', 'get_credenti', 'new_axi', 'expected_arg', '7', 'one-vs-rest', 'client=', 'margin-bas', 'colormap=', 'drbd9', 'expiration_polici', 'x_3', 'x_3', '`examples/howto/patch_app', '`event`', 'journal={iclr}', 'auth_secret', 'tokenembedding`', 'sep``', '**kwarg', '--list-loc', 'branch/tag', 'new_valu', 'reflect', '/mydatacenter/vm/myvmfold', 'peform', 'batched_reduce_s', '在实盘中', 'non-http', 'encoding``', 'fog', '<config>', 'network_security_groups_list', 'ih', 'com/blog/2009/11/trying-out-retry-decorator-python/', 'wk2', 'get_zon', 'webserver/security/authentication/anonymousauthent', 'myapach', 'atla', '380', 'xml_path', 'name_map', '``msg``', 'rolling_window', 'pcmk_host_map=node1', 'tar/zip', 'servicegroup_server_up', 'mask', 'fielddescriptor', 'plot2', 'event_valu', 'autobuilding=tru',

In [96]:
i = 0
for test in test_dataloader:
    seq_in = train_in[:,0,:]
    struct_in =train_in[:,1,:]
    target_tensor = train_in[:,2,:]
    out = ActorNetwork(seq_in, struct_in)
    _, topi = out.topk(1)
    decoded_ids = topi.squeeze()
    break

# 4. Modifications 

In [75]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()


# Solver test